In [1]:
import requests
import pandas as pd
from pymongo import MongoClient
from datetime import datetime
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact


# Define the API endpoint and parameters
url = "https://api.fda.gov/food/enforcement.json?search=report_date:[20200101+TO+20241005]&limit=500"

# Send a GET request to the API
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    recalls = data.get('results', [])
    recall_df = pd.DataFrame(recalls)

    clean_df = recall_df.rename(columns={
        "status": "Status", 
        "city": "City",
        "state": "State",
        "country":"Country",
        "classification":"Classification",
        "openfda":"Open FDA",
        "product_type":"Product Type",
        "event_id":"Event ID",
        "recalling_firm":"Recalling Firm",
        "address_1":"Address 1",
        "address_2":"Address 2",
        "postal_code":"Postal Code",
        "voluntary_mandated":"Voluntary Mandated",
        "initial_firm_notification":"Initial Firm Notification",
        "distribution_pattern":"Distribution Pattern",
        "recall_number":"Recall Number",
        "product_description":"Product Description",
        "product_quantity":"Product Quantity",
        "reason_for_recall":"Reason for Recall",
        "recall_initiation_date":"Recall Initiation Date",
        "center_classification_date":"Center Classification Date",
        "termination_date":"Termination Date",
        "report_date":"Report Date",
        "code_info":"Code Info",
        "more_code_info":"More Code Info",
    })

    # Convert DataFrame to dictionary format for MongoDB
    recall_records = clean_df.to_dict("records")

    # Modify records to ensure the correct structure
    for record in recall_records:
        record['recall_number'] = record.get('recall_number', '')  # Primary key
        record['recall_initiation_date'] = datetime.strptime(record.get('recall_initiation_date', '1900-01-01'), '%Y%m%d') if record.get('recall_initiation_date') else None
        record['recall_classification'] = record.get('classification', '')  # Classification as VARCHAR(50)
        record['status'] = record.get('status', '')  # Status as VARCHAR(50)
        record['product_description'] = record.get('product_description', '')  # Description as TEXT
        record['code_info'] = record.get('code_info', '')  # Code info as TEXT
        record['recalling_firm'] = record.get('recalling_firm', '')  # Firm as VARCHAR(50)
        record['reason_for_recall'] = record.get('reason_for_recall', '')  # Reason as TEXT
        record['distribution_pattern'] = record.get('distribution_pattern', '')  # Pattern as TEXT
        record['state'] = record.get('state', '')  # State as VARCHAR(50)
        record['report_date'] = datetime.strptime(record.get('report_date', '1900-01-01'), '%Y%m%d') if record.get('report_date') else None
        record['voluntary_mandated'] = record.get('voluntary_mandated', '')  # Voluntary/Mandated as VARCHAR(50)
        record['event_id'] = record.get('event_id', '')  # Event ID as VARCHAR(50)

    # Connect to MongoDB
    client = MongoClient('mongodb://localhost:27017/')  # Or use your MongoDB URI
    db = client['fda_recall_data']  # Database name
    collection = db['recalls']  # Collection name

    # Insert modified data into MongoDB
    collection.insert_many(recall_records)

    # Example: Query data back from MongoDB
    recalls_from_db = list(collection.find())

    # Convert back to DataFrame if needed
    db_df = pd.DataFrame(recalls_from_db)

In [2]:

# Filter the DataFrame to include only recalls in the US (made a copy to avoid SettingWithCopyWarning)
us_recall_df = clean_df[clean_df['Country'] == "United States"].copy()

# Add a count column for each recall for visualization purposes
us_recall_df['Recall Count'] = 1

# Sort the top 15 recalling firms based on the number of recalls in the US
top_15_firms = (
    us_recall_df['Recalling Firm']
    .value_counts()
    .nlargest(15)
    .reset_index()
)

# Remame the columns in the identify the recall count for each
top_15_firms.columns = ['Firm Name', 'Recall Count']

# Create a list for dropdown options with the name of each firm and their recall counts
firm_options = [
    f"({row['Recall Count']}) {row['Firm Name']}"
    for _, row in top_15_firms.iterrows()
]

# Create dropdown widget for selecting a firm, including "None" as an option
firm_widget = widgets.Dropdown(
    options=[None] + firm_options,
    description='Select Firm:',
    value=None
)

# Function to create a Sunburst chart
def create_sunburst_chart(firm_with_count):
    # Extract firm name from dropdown selection
    if firm_with_count is None:
        print("Select a firm from the dropdown menu.")
        return

    try:
        # Extract the actual firm name by splitting the string
        firm = firm_with_count.split(") ")[1]
    except IndexError:
        print("Error extracting firm name from the dropdown selection.")
        return

    # Filter the data based on the selected firm
    filtered_df = us_recall_df[us_recall_df['Recalling Firm'] == firm]

    # Ensure there's data to plot
    if filtered_df.empty:
        print("No data available for the selected firm.")
        return

    # Create Sunburst chart using Plotly with customized hover information
    try:
        # Create Sunburst chart with customized hover data
        fig = px.sunburst(
            filtered_df,
            path=['Recalling Firm', 'Product Type', 'Product Description'],
            values='Recall Count',  # Use the count column for visualization
            title=f"Top Products Recalled by {firm}",
            color='Product Type',
            hover_data={
                "Classification": True,
                "Product Description": True,
                "State": True
            }
        )
        
        # Update hover template to display only the firm name
        fig.update_traces(
            hovertemplate="<br>".join([
                "Firm: " + firm,
                "Classification: %{customdata[0]}",
                "Product Description: %{customdata[1]}",
                "State: %{customdata[2]}"
            ])
        )
        
        fig.show()
        
        # Create HTML file
        fig.write_html("./output_data/fda_recalls_by_recall_firm.html")
        
        
    except Exception as e:
        print(f"An error occurred while creating the Sunburst chart: {e}")

# Create the interactive UI
interact(
    create_sunburst_chart,
    firm_with_count=firm_widget
)


interactive(children=(Dropdown(description='Select Firm:', options=(None, '(33) FRESH IDEATION FOOD GROUP, LLC…

<function __main__.create_sunburst_chart(firm_with_count)>